# Text Generator at 'Character' level <br>
### This model generates random names. This is an AI naming your baby (Beware, they are coming for us!)<br>
#### So if you have a new baby, or your parents name you 'X Æ A-Xii' feel free to use this.

*comments would have explanation of the code*

In [1]:
#Basic imports

import torch
import torch.nn as nn
import string
import random 
import unidecode  #to read text file per character 
from torch.utils.tensorboard import SummaryWriter

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #to run our model on GPU
print(device)

cuda


In [12]:
all_characters = string.printable #obtaining all the characters used by humans to name humans
print("All Characters -> "+ all_characters)
n_characters = len(all_characters)
print("Total number of characters: ", n_characters)

All Characters -> 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

Total number of characters:  100


In [20]:
file = unidecode.unidecode(open('names.txt').read()) #reading our data file, file = 'M i c h a e l \n C .........'
file[:250]

'Michael\nChristopher\nJessica\nMatthew\nAshley\nJennifer\nJoshua\nAmanda\nDaniel\nDavid\nJames\nRobert\nJohn\nJoseph\nAndrew\nRyan\nBrandon\nJason\nJustin\nSarah\nWilliam\nJonathan\nStephanie\nBrian\nNicole\nNicholas\nAnthony\nHeather\nEric\nElizabeth\nAdam\nMegan\nMelissa\nKevin\nSt'

In [19]:
class RNN(nn.Module): # Can be called the fundamental building block of the model.
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size) # This is something the RNN is going to learn. Ex = e, where E is the embedding matrix, x is the input and e is the embedded vector generated.

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))

        return out, (hidden, cell)

    def init_hidden(self, batch_size): #just to innitialize hidden and cell for the LSTM
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell


In [33]:
class Generator(nn.Module): # Generator does most of our work. As the name suggests it would generate names and do the training.
    def __init__(self):
        super().__init__()
        self.chunk_len = 250 # length of output characters "b, d, /n, ...."
        self.num_epochs = 5000 
        self.batch_size = 1
        self.print_every = 100
        self.hidden_size = 256
        self.num_layers = 3
        self.lr = 0.003

    def char_tensor(self, string): # map a string to a tensor 
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c]) # returns indices of of the characters 

        return tensor

    def get_random_batch(self):
        start_idx = random.randint(0, len(file)-self.chunk_len) # ensuing that start_idx would have atleast self.chunk_len after it 
        end_indx = start_idx + self.chunk_len + 1
        text_str = file[start_idx:end_indx] 
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)

        for i in range (self.batch_size):
            text_input[i,:] = self.char_tensor(text_str[:-1])
            text_target[i, :] = self.char_tensor(text_str[1:]) #the prediction is the next character

        return text_input.long(), text_target.long()

    def generate(self, initial_str = 'A', predict_len = 100, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        predicted = initial_str

        for p in range(len(initial_str) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)


        last_char = initial_input[-1]
         
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)

        return predicted




        

    def train(self):
        self.rnn = RNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device)
        
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        
        writer = SummaryWriter(f'runs/names0')

        print('====> Starting training')

        for epoch in range (1, self.num_epochs + 1):
            inp, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size = self.batch_size)

            self.rnn.zero_grad()

            loss = 0
            inp = inp.to(device)
            #print('inp: ', inp.shape)
            target = target.to(device)
            #print('target: ', target.shape)
            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:, c], hidden, cell)
                #print('inp[:, c]: ', inp[:, c])
                #print('output: ', output.shape)
                #print('target[:, c] ', target[:, c].shape)
                loss += criterion(output, target[:, c])
                
    
            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f'Loss: {loss}')
                

            writer.add_scalar('Training loss', loss, global_step=epoch)

        print(self.generate())


In [34]:
gennames = Generator()
gennames.train()

====> Starting training
Loss: 2.594562744140625
Loss: 2.26596728515625
Loss: 2.24347119140625
Loss: 2.255954833984375
Loss: 2.22024169921875
Loss: 2.1541376953125
Loss: 2.3362548828125
Loss: 2.211706787109375
Loss: 2.313609375
Loss: 2.091497314453125
Loss: 1.9580394287109375
Loss: 1.7014146728515624
Loss: 2.125843994140625
Loss: 1.9344033203125
Loss: 1.9112117919921876
Loss: 2.284055908203125
Loss: 1.793951171875
Loss: 2.08261279296875
Loss: 1.869141845703125
Loss: 1.949829345703125
Loss: 2.168683837890625
Loss: 2.05549267578125
Loss: 2.06977880859375
Loss: 1.9278770751953125
Loss: 2.157693603515625
Loss: 1.9790606689453125
Loss: 1.9210311279296874
Loss: 1.707993896484375
Loss: 1.955552490234375
Loss: 1.96122216796875
Loss: 1.7981553955078124
Loss: 1.7454229736328124
Loss: 1.7346317138671874
Loss: 1.778651611328125
Loss: 1.753957275390625
Loss: 1.562507080078125
Loss: 1.58137109375
Loss: 1.7126717529296875
Loss: 1.774057861328125
Loss: 1.70461767578125
Loss: 1.605557861328125
Loss: 1.6